In [ ]:
#dataset = pd.read_csv('mnist_test.csv')

In [12]:
import numpy as np
import pandas as pd

qtInter,qtOut,qtInput,e,alpha,epoca = 100,10,784,0,0.1,0
dataset = pd.read_csv('mnist_train.csv')
v = np.random.randn(qtInput, qtInter) / np.sqrt(qtInput)
w = np.random.randn(qtInter, qtOut) / np.sqrt(qtInter)
ativacao = 'sig'

In [13]:
grafico = np.array([[0]*2])

In [14]:
def trataLinha(inp):
    retorno = np.array(inp)
    return retorno/255

def targetVetor(labels, num_classes=10):
    retTarget = np.eye(num_classes)[labels]
    return retTarget

def inputZ(inp):
    inX = np.zeros(qtInter)
    for j in range (qtInter):
        inX[j] = np.sum(inp * np.array(v[:,j]))
    return inX

def inputY(inp,pesos):
    inX = np.zeros(qtOut)
    for k in range (qtOut):
        inX[k] = np.sum(inp * np.array(pesos[:,k]))
    return inX

def deltainJ (dK,peso):
    inJ = np.zeros(qtInter)
    for j in range (qtInter):
        for k in range (qtOut):
            inJ[j] += dK[k] * peso[j][k]
    return inJ

def funcAtivacao(x,func) :
    if (func == 'relu') : 
        return np.maximum(x,0) 
    elif (func == 'sig') : 
        return (1/(1+np.exp(-x)))
    elif (func == 'tanh'):
        return np.tanh(x)

def derivada(x,func):        
    if (func == 'relu') : 
        return (0 if x<0 else 1)
    elif (func == 'sig') : 
        return (x * (1 - x))
    elif (func == 'tanh') :
        return (1 - np.power(x,2))

def verificaAcerto(tar,out):
    indTar = np.unravel_index(np.argmax(tar, axis=None), tar.shape)
    indOut = np.unravel_index(np.argmax(out, axis=None), out.shape)
    if (indTar == indOut):
        return True
    else :
        return False

def deltaK(targetK,Yk,YinK,func) :
    dK = np.zeros(qtOut)
    for i in range (qtOut) :
        erro = targetK[i] - Yk[i]
        deriv = derivada(YinK[i],func)
        dK[i] = erro * deriv
    return dK

def deltaJ(inJ,inZ,func):
    delJ = np.zeros(qtInter)
    for j in range (qtInter):
        deriv = derivada(inZ[j],func)
        delJ[j] = inJ[j] * deriv
    return delJ

def deltaW(dK,Ze):
    dW = np.empty_like(w)
    for j in range(qtInter):
        for k in range(qtOut):
            dW[j][k] = alpha * dK[k] * Ze[j]
    return dW

def deltaV(linha,dJ):
    dV = np.empty_like(v)
    for i in range(qtInput):
        for j in range (qtInter):
            dV[i][j] = alpha * dJ[j] * linha[i]
    return dV

In [15]:
def algoritmo(isTreino,qtEpoca):
    global epoca
    for i in range (qtEpoca):
        readLine(dataset,isTreino)
        epoca+=1

In [16]:
def readLine(dSet,isTreino):
    global e
    r,e = 1,0
    for row in dSet.itertuples(index=False):
        linha = list(row)
        target = targetVetor(linha.pop(0))
        linha = trataLinha(linha)
        forward(linha,isTreino,target,r)
        r+=1
        
def forward(linha,isTreino,target,r):
    global e
    global grafico
    inZ = inputZ(linha)
    Z = funcAtivacao(inZ,ativacao)
    inY = inputY(Z,w)
    Y = funcAtivacao(inY,ativacao)
    if (not verificaAcerto(target,Y)):
        e+=1
    if isTreino:
        backPropagation(inZ,Z,inY,Y,target,linha)
    a = 100 - ((e/r)*100)
    if (r!=0 and r%20 == 0):
        print (target)
        print (Y)
        print (f'Linha: {r} Epoca: {epoca} Erros: {e}  Acerto: {a} %' + '\n' + '_________' )
        grafico = np.append(grafico,[[r,a]],axis=0)
    
def backPropagation(inZ,Z,inY,Y,target,linha):
    delK,inJ,delJ =[],[],[]
    delK = deltaK(target,Y,Y,ativacao) 
    inJ  = deltainJ(delK,w)
    delJ = deltaJ(inJ,Z,ativacao)
    atualizaPesos(delK,Z,linha,delJ)

In [17]:
def atualizaPesos(delK,Z,linha,delJ):
    global w
    global v
    correcaoW = deltaW(delK,Z)
    correcaoV = deltaV(linha,delJ)
    w = corrigePeso(w,correcaoW)
    v = corrigePeso(v,correcaoV)
    
def corrigePeso(peso,delta):
    return np.add(peso,delta)

In [18]:
algoritmo(True,1)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.12503863 0.19014754 0.15969324 0.16802285 0.13715877 0.1415232
 0.1679147  0.1508822  0.15187222 0.1270293 ]
Linha: 20 Epoca: 0 Erros: 18  Acerto: 10.0 %
_________
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0.14297028 0.11743854 0.11973602 0.12385375 0.10187773 0.09431482
 0.13955014 0.12008569 0.09504524 0.12510165]
Linha: 40 Epoca: 0 Erros: 37  Acerto: 7.5 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.11200474 0.09104548 0.07141331 0.1338226  0.11723651 0.08220207
 0.08943723 0.11088211 0.11663326 0.19439539]
Linha: 60 Epoca: 0 Erros: 55  Acerto: 8.333333333333343 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.17317513 0.22628022 0.064851   0.08352986 0.09572983 0.07410799
 0.10574546 0.0918858  0.08375537 0.09839708]
Linha: 80 Epoca: 0 Erros: 71  Acerto: 11.25 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.14111973 0.11903696 0.062756   0.1059612  0.11910206 0.06706999
 0.12126222 0.14458757 0.1120531  0.10012664]
Linha: 100 Epoca: 0 Erros: 88  Acerto: 12.0

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0.20774557 0.06107726 0.04906652 0.13590044 0.05700254 0.12502938
 0.13676034 0.17642968 0.20428999 0.14436917]
Linha: 800 Epoca: 0 Erros: 541  Acerto: 32.375 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.09103996 0.04691168 0.08797102 0.08154339 0.08968683 0.09838621
 0.0903823  0.3700121  0.11099074 0.22178545]
Linha: 820 Epoca: 0 Erros: 550  Acerto: 32.92682926829268 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.03435212 0.19107632 0.07086438 0.04985239 0.10179083 0.11774255
 0.06188818 0.43401082 0.10727261 0.21785842]
Linha: 840 Epoca: 0 Erros: 560  Acerto: 33.33333333333334 %
_________
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.57087752 0.01649045 0.08759637 0.14573952 0.23301592 0.16177524
 0.12739292 0.11754226 0.0660525  0.06920069]
Linha: 860 Epoca: 0 Erros: 569  Acerto: 33.837209302325576 %
_________
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0.06535874 0.13241073 0.1246958  0.20365079 0.19142427 0.06946464
 0.10252554 0.089717   0.10919507 0.07667011]
Linha

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.0056295  0.73859256 0.27614591 0.11666269 0.05875359 0.09951971
 0.03164523 0.08472311 0.0975187  0.0780357 ]
Linha: 1580 Epoca: 0 Erros: 837  Acerto: 47.0253164556962 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.09429794 0.03945944 0.05775702 0.587484   0.02462365 0.17999567
 0.02084808 0.07571743 0.13242663 0.11039642]
Linha: 1600 Epoca: 0 Erros: 844  Acerto: 47.25 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.06578155 0.01954599 0.02002065 0.04996948 0.07708412 0.06863113
 0.01484901 0.80452303 0.03774047 0.38912121]
Linha: 1620 Epoca: 0 Erros: 850  Acerto: 47.53086419753087 %
_________
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0.02585229 0.03379958 0.0220963  0.06557228 0.07843614 0.06675544
 0.01131811 0.70959692 0.04520134 0.32446779]
Linha: 1640 Epoca: 0 Erros: 856  Acerto: 47.80487804878049 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.01057113 0.04492986 0.02611373 0.04196135 0.22935351 0.08073028
 0.04748373 0.21981281 0.10741605 0.43657019]
Linh

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.01159793 0.15964712 0.33954126 0.27501077 0.0441422  0.13022382
 0.00297573 0.01951056 0.25110173 0.02867881]
Linha: 2340 Epoca: 0 Erros: 1008  Acerto: 56.92307692307692 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.01317951 0.06106584 0.64185719 0.43046117 0.13589103 0.06902932
 0.01022896 0.00451993 0.23309367 0.01116713]
Linha: 2360 Epoca: 0 Erros: 1012  Acerto: 57.11864406779661 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.01135124 0.28565681 0.18598163 0.10085383 0.02565944 0.04539762
 0.00598115 0.07785734 0.22486539 0.08644657]
Linha: 2380 Epoca: 0 Erros: 1018  Acerto: 57.226890756302524 %
_________
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0.01891699 0.01676331 0.88296915 0.05196225 0.06417183 0.02620584
 0.08768969 0.00301238 0.43730295 0.01222833]
Linha: 2400 Epoca: 0 Erros: 1019  Acerto: 57.54166666666667 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.0035364  0.37424019 0.2371325  0.04284762 0.16856705 0.07721427
 0.00639502 0.019556   0.3940862

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.00615988 0.01826693 0.0066611  0.00653413 0.22174046 0.06738586
 0.04942227 0.08860459 0.16108229 0.62625325]
Linha: 3100 Epoca: 0 Erros: 1147  Acerto: 63.0 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0.0159067  0.09151805 0.15970569 0.40770566 0.0125971  0.06503801
 0.01651038 0.01476083 0.17409979 0.00904596]
Linha: 3120 Epoca: 0 Erros: 1150  Acerto: 63.141025641025635 %
_________
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0.01584035 0.00316154 0.06040788 0.00143118 0.13448017 0.07948344
 0.88785586 0.02305339 0.01747694 0.12688555]
Linha: 3140 Epoca: 0 Erros: 1153  Acerto: 63.28025477707007 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.02832024 0.01295571 0.07113793 0.77835073 0.0092377  0.14487203
 0.00826526 0.01356473 0.34178388 0.02005406]
Linha: 3160 Epoca: 0 Erros: 1155  Acerto: 63.449367088607595 %
_________
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0.00679806 0.02371314 0.96335419 0.04806674 0.02903452 0.00886487
 0.11350755 0.01313113 0.08239756 0.0070726 

...

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[9.92803833e-01 4.15023544e-07 3.07270640e-03 1.31204926e-04
 1.09595385e-04 1.53440353e-02 6.97357989e-04 2.57449088e-03
 4.00133873e-04 3.38256610e-05]
Linha: 57360 Epoca: 0 Erros: 5809  Acerto: 89.87273361227336 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[4.73085042e-03 1.48991729e-05 6.92250225e-02 1.89467199e-04
 5.70054626e-05 1.74625032e-02 8.81172399e-05 1.83884432e-03
 9.48053521e-01 7.74593641e-04]
Linha: 57380 Epoca: 0 Erros: 5809  Acerto: 89.87626350644824 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[1.36683881e-04 1.61907637e-03 8.69055197e-03 9.96312215e-01
 3.53887563e-05 9.37833811e-03 2.23896921e-05 3.14719436e-05
 3.19312558e-02 3.92433406e-05]
Linha: 57400 Epoca: 0 Erros: 5811  Acerto: 89.87630662020906 %
_________
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[1.60406725e-03 2.07718539e-06 3.90546614e-03 6.06900188e-07
 2.31065700e-02 3.14536164e-05 9.91329786e-01 1.30727560e-03
 1.27646143e-04 9.48677503e-03]
Linha: 57420 Epoca: 0 Erros: 5812  Ac

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[4.81777068e-04 2.05243836e-04 9.99778692e-01 1.44018026e-03
 5.59643338e-05 9.56075509e-05 1.04496284e-02 4.64205001e-06
 2.74780012e-03 2.30497663e-06]
Linha: 58000 Epoca: 0 Erros: 5840  Acerto: 89.93103448275862 %
_________
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[1.54327265e-03 4.60081871e-04 9.99721391e-01 2.46412355e-03
 1.62835926e-04 2.97091057e-04 4.85525816e-03 5.18081079e-06
 2.30077305e-03 3.00331760e-06]
Linha: 58020 Epoca: 0 Erros: 5840  Acerto: 89.93450534298518 %
_________
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[4.20826237e-02 9.16467296e-06 2.57207078e-05 7.53238305e-04
 2.14219787e-05 9.94142570e-01 7.44963426e-05 1.97802958e-04
 2.40105974e-03 7.34586547e-04]
Linha: 58040 Epoca: 0 Erros: 5841  Acerto: 89.93625086147485 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[7.93131373e-05 2.79567307e-05 1.08101479e-04 1.27614879e-02
 1.14462539e-02 6.55906703e-04 1.22275230e-03 4.39138587e-03
 5.19481579e-03 9.82773064e-01]
Linha: 58060 Epoca: 0 Erros: 5842  Ac

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[1.26692344e-04 1.69023196e-05 3.19838383e-05 8.62750466e-04
 2.89437083e-02 6.24596053e-03 1.74715567e-04 1.27366988e-02
 7.94701395e-03 8.85339724e-01]
Linha: 58640 Epoca: 0 Erros: 5859  Acerto: 90.00852660300137 %
_________
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[9.79462290e-01 1.79439095e-07 1.97744207e-03 2.37666414e-03
 1.94430734e-04 2.09364300e-02 1.79459570e-03 2.76592812e-03
 4.20773172e-03 8.57440630e-03]
Linha: 58660 Epoca: 0 Erros: 5861  Acerto: 90.00852369587453 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[2.59458366e-03 3.81203781e-03 1.88119672e-02 1.72357946e-03
 9.76889983e-05 1.94384404e-02 2.44611510e-04 4.37584402e-05
 9.89339185e-01 5.51892985e-04]
Linha: 58680 Epoca: 0 Erros: 5862  Acerto: 90.01022494887526 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[2.29535604e-04 1.71962713e-05 9.48863905e-05 9.38904331e-05
 1.27166555e-02 1.94670111e-03 5.47257908e-04 2.11859492e-02
 4.73791065e-03 9.40763172e-01]
Linha: 58700 Epoca: 0 Erros: 5862  Ac

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[1.92204995e-04 5.71105611e-06 9.28660645e-04 3.25990727e-04
 3.81003524e-05 1.58755622e-03 1.28836421e-05 9.84858352e-01
 7.08858040e-04 1.31845872e-02]
Linha: 59280 Epoca: 0 Erros: 5874  Acerto: 90.0910931174089 %
_________
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[6.78510029e-04 9.53201508e-05 8.17824886e-05 9.74667513e-01
 6.55539613e-05 3.23600672e-03 9.94447795e-07 1.74419625e-02
 5.23246428e-03 1.24817251e-01]
Linha: 59300 Epoca: 0 Erros: 5877  Acerto: 90.0893760539629 %
_________
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[9.87206707e-01 6.75673882e-07 4.85333251e-04 2.85156898e-04
 3.49747351e-05 1.49642191e-02 8.13177446e-04 2.95138338e-04
 1.13949706e-04 2.62148761e-05]
Linha: 59320 Epoca: 0 Erros: 5880  Acerto: 90.08766014834794 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[2.20981027e-04 6.89962272e-01 3.13761442e-06 3.10847866e-02
 3.13408980e-03 9.86949969e-02 8.53654880e-05 1.01992299e-03
 2.86518974e-01 1.27709588e-03]
Linha: 59340 Epoca: 0 Erros: 5880  Acer

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[2.57959989e-04 4.76147137e-01 1.12227703e-02 3.06271722e-02
 7.85541590e-03 1.15581224e-02 5.54739786e-06 9.95276243e-05
 7.21467072e-02 1.29303062e-03]
Linha: 59920 Epoca: 0 Erros: 5908  Acerto: 90.14018691588785 %
_________
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[3.01892958e-03 3.01046716e-06 1.37082923e-04 3.36717449e-04
 2.95497841e-03 2.49056488e-02 1.79437494e-04 1.11243525e-01
 1.54171864e-04 5.78494004e-01]
Linha: 59940 Epoca: 0 Erros: 5910  Acerto: 90.14014014014015 %
_________
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[1.75803671e-03 1.57765492e-05 6.43893629e-03 6.18302457e-06
 1.97196845e-02 6.92708102e-05 9.92872751e-01 9.38529779e-06
 5.63748099e-04 3.20542933e-03]
Linha: 59960 Epoca: 0 Erros: 5911  Acerto: 90.14176117411608 %
_________
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[2.00223467e-06 9.93415529e-01 3.60229226e-03 9.78594093e-03
 5.23994943e-03 2.68043448e-03 1.21259814e-04 3.90626872e-03
 3.07352507e-03 1.02718970e-03]
Linha: 59980 Epoca: 0 Erros: 5911  Ac